In [1]:
##SciBERT: https://github.com/allenai/scibert
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, AutoTokenizer, BertTokenizerFast, BertModel, BertTokenizer
import os
import math

In [2]:
import torch
print(torch.version.cuda)  # Should print a version if CUDA-enabled
print(torch.cuda.is_available())  # Should be True

12.8
True


In [3]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 5090


# Making bert embeddings

In [8]:
##Use SciBERT instaed of BERT
# import SciBERT-base pretrained model

# Set HuggingFace cache path

cache_path = r"F:\ai_playground"
model_name = "allenai/scibert_scivocab_uncased"

model = AutoModel.from_pretrained(model_name, cache_dir=cache_path)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_path)



##BERT
# bert = AutoModel.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

F:\One Drives\OneDrive - Nanyang Technological University\Academics\!win_dev\mae_fyp\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in F:\ai_playground\models--allenai--scibert_scivocab_uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is 

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [9]:
from src.file_handling import file_location

folder_path = file_location.FileLocation()
root_path = folder_path.root
data_path = root_path.parent.parent / 'fyp' / 'data'

asme_path = file_location.FolderPathOfASME(data_path)
jmd_all_updated_cleaned_excel_path = asme_path.asme_jmd / 'jmd_all_updated cleaned.xlsx'

orig_data = pd.read_excel(jmd_all_updated_cleaned_excel_path)
print('Number of articles: {:,}\n'.format(orig_data.shape[0]))

Number of articles: 5,591



In [10]:
## exclude a part of papers according to issue_section info
remove_set = ["Issue Section:\nLetters to the Editor", "Issue Section:\nEditorial", "Issue Section:\nIn Memoriam", 
              "Issue Section:\nDiscussions", "Issue Section:\nErrata", "Issue Section:\nBook Reviews",
              "Issue Section:\nReports", "Issue Section:\nGuest Editorial", "Issue Section:\nAnnouncements", 
              "Issue Section:\nCommentary", "Issue Section:\nRetrospectives", "Issue Section:\nBook Review",
              "Issue Section:\nObituary", "Issue Section:\nErratum", "Issue Section:\nEditor's Note", 
              "Issue Section:\nIntroduction", "Issue Section:\nEditorials", "Issue Section:\nDiscussion", 
              "Issue Section:\nSurvey", "Issue Section:\nAnnouncement"]

data = orig_data[~orig_data['issue_section'].isin(remove_set)]
len(data)

##munually removed a few papers after this

5121

In [7]:
##reload data
datax = pd.read_excel('data with cluster.xlsx') 
data = orig_data[orig_data['doi'].isin(list(datax['doi']))]

'''
Manually clean the text data: integrate the results from 2 pdf2text approaches and put the main-body text in columns:
['text 0', 'text 1', 'text 2']
'''
data = data.astype({'title':str, 'abstract':str, 'keywords':str, 'text 0':str, 'text 1':str, 'text 2':str})
info = {}
info['lst1'] = list(data['title'])
info['lst2'] = list(data['abstract'])
info['lst3'] = list(data['keywords'])
info['lst4'] = list(data['text 0'])
info['lst5'] = list(data['text 1'])
info['lst6'] = list(data['text 2'])

##use title and abstract for analysis
# var = 'TA'
##use title, abstract, and main-body text for analysis
var = 'FULL'

if var == 'TA':
    n = 2
else:
    n = 6
    
text = []
for i in range(0,len(data)):
    txt = [info[key][i] for key in list(info.keys())[0:n]]
    ##clean keywords
    txt[2] = txt[2].replace('Keywords:\n', '').replace('\n', '')
    ##join the elements
    txt1 = ' '.join(txt[0:3]).strip()
    txt2 = ''.join(txt[3:6]).strip()
    txt3 = ' '.join([txt1, txt2]).strip()
    text.append(txt3)
    
data['text'] = text
data = data[data.text.notnull()]
len(data)

FileNotFoundError: [Errno 2] No such file or directory: 'data with cluster.xlsx'

In [ ]:
# text

In [ ]:
##only keep the relevant columns
data = data[['title', 'abstract', 'doi', 'year', 'issue',
       'citation_info', 'authors', 'author affiliation', 'pdf_link',
       'issue_section', 'topics', 'cited times', 'keywords', 'reference',
       'related article', 'source of related articles', 'text 0', 'text 1', 'text 2']]

print(len(data))

# Generate BERT embeddings

In [ ]:
##split long text into multiple piece
new_text = {}
new_text['ID'] = []
new_text['text'] = []
m = 0
for item in text:
    txt = item.split()
    n = int(len(text)/500) + 1
    for i in range(0,n):
        txt1 = txt[500*i:min(500*(i+1),len(txt))]
        txt2 = ' '.join(txt1).strip()
        new_text['ID'].append(m)
        new_text['text'].append(txt2)
    m += 1
new_text = pd.DataFrame(new_text)

new_text.to_excel('truncated text.xlsx')

In [ ]:
text1 = list(new_text['text'])
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
#     train_text.tolist(),
    text1,
    max_length = 500,
    pad_to_max_length=True,
    truncation=True
)

# get length of all the messages in the train set
seq_len = [len(i.split()) for i in text1]
pd.Series(seq_len).hist(bins = 30)

In [ ]:
## convert lists to tensors
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 64

# wrap tensors
train_data = TensorDataset(train_seq, train_mask)

# sampler for sampling the data during training
train_sampler = SequentialSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
device = torch.device("cuda")
bert = bert.to(device)

In [ ]:
# iterate over batches

cls_all = []
for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
        print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]

    sent_id, mask = batch 

    # get model predictions for the current batch
    cls = bert(sent_id, mask,return_dict=False)[1]
    cls_all.append(cls)

In [ ]:
embedding = cls_all[0].cpu().detach().numpy()
for i in cls_all[1:]:
    embedding = np.concatenate((embedding, i.cpu().detach().numpy()), axis=0)
    
np.savetxt('embeddings of truncated texts', embedding)

In [ ]:
##integrate embeddings of multiple truncated texts into 1 as the final embeddings of the long text using attention

lst = list(set(new_text['ID']))
new_emb = np.zeros((len(data),768))
for i in lst:
    tmp = new_text[new_text['ID'] == i]
    idx = list(tmp.index)
    emb = embedding[idx]
    wgt_sum = 0
    wgt = [0]*len(emb)
    for a in range(0,len(emb)):
        for b in range(0,len(emb)):
            wgt_sum += np.dot(emb[a], emb[b])
            wgt[a] += np.dot(emb[a], emb[b])
    wgt1 = [j/wgt_sum for j in wgt]
    for a in range(0,len(emb)):
        new_emb[i] += wgt1[a]*emb[a]
        
np.savetxt('embeddings of long texts', new_emb)

# clustering

In [ ]:
data['emb'] = list(new_emb)

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,var,NUM_CLUSTERS = 8):

    X = np.array(data[var].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster_'+str(NUM_CLUSTERS)] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster_'+str(NUM_CLUSTERS)].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

In [ ]:
from scipy.spatial import distance_matrix

def distance_from_centroid(row,var):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row[var]], [row['centroid'].tolist()])[0][0]

In [ ]:
#clusters data using num_clusters 
def get_cluster_question(input, num_clusters):
    new_cluster_data, new_cluster_id = clustering_question(input, var, num_clusters)
    new_cluster_data['cluster_id'] = new_cluster_id
    new_cluster_data['distance_from_centroid'] = new_cluster_data.apply(distance_from_centroid, axis=1)
    return new_cluster_data

# Find ideal number of clusters

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import math
import matplotlib.style as style
import plotly
import plotly.graph_objs as go
from sklearn.decomposition import PCA

In [ ]:
## find the optimal reduced dimensions based on explained variance
n = 100
pca_n = PCA(n_components=n)
emb_pca = pca_n.fit(new_emb)
exp_var_pca_n = pca_n.explained_variance_ratio_
cumul_sum_eigenvals = np.cumsum(exp_var_pca_n)

plt.bar(range(0,len(exp_var_pca_n)), exp_var_pca_n, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cumul_sum_eigenvals)), cumul_sum_eigenvals, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
no = next(idx for idx, value in enumerate(cumul_sum_eigenvals) if value > 0.9)

'''
9 for full-texts and 42 for TAs. That is full-texts can be represented by lower-dimention data.
Full-texts are more similar to each other? More noise?
'''
print ("Dimentions: ", no)
print ("Explained variance: ",cumul_sum_eigenvals[no])

In [ ]:
##get low-dimensional embeddings
pca = PCA(n_components=no)
pca.fit(new_emb)
x = pca.fit_transform(new_emb)

data['emb_reduced'] = x.tolist()

##get 2-dimensional embeddings for visualization
pca = PCA(n_components=2)
pca.fit(new_emb)
x = pca.fit_transform(new_emb)

data['X'] = x[:,0]
data['Y'] = x[:,1]

In [ ]:
##silhouette analysis: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
def find_opt_clusters_4(data, embedding, min_num, max_num):
    stored_clusters = {}
    range_n_clusters = list(range(min_num, max_num))
    silh_avg_n_clusters = []
  
    for n_clusters in range_n_clusters:
        fig, ax = plt.subplots(1,2,figsize=(15,5))

        # Run the kmeans algorithm using "emb_reduced" or "emb"
        cluster_data, cluster_labels = clustering_question(data,'emb', n_clusters)
        stored_clusters[n_clusters] = (cluster_data, cluster_labels)

        # density & separation of formed clusters
        silh_avg = silhouette_score(embedding, cluster_labels)
        print("For n_clusters =", n_clusters, "The average silhouette_score is :", silh_avg)
        silh_avg_n_clusters.append(silh_avg)

        centroids  = cluster_data['centroid']

        # get silhouette

        silhouette_vals = silhouette_samples(embedding,cluster_labels) #silhouette_vals

        # silhouette plot
        y_ticks = []
        y_lower = y_upper = 0
        for i,cluster in enumerate(np.unique(cluster_labels)):
            cluster_silhouette_vals = silhouette_vals[cluster_labels ==cluster]
            cluster_silhouette_vals.sort()
            y_upper += len(cluster_silhouette_vals)

            ax[0].barh(range(y_lower,y_upper),
            cluster_silhouette_vals,height =1);
            ax[0].text(-0.03,(y_lower+y_upper)/2,str(i+1))
            y_lower += len(cluster_silhouette_vals)

            # Get the average silhouette score 
            avg_score = np.mean(silhouette_vals)
            ax[0].axvline(avg_score,linestyle ='--',
            linewidth =2,color = 'green')
            ax[0].set_yticks([])
            ax[0].set_xlim([-0.1, 1])
            ax[0].set_xlabel('Silhouette coefficient values')
            ax[0].set_ylabel('Cluster labels')
            ax[0].set_title('Silhouette plot for the various clusters');

            # scatter plot of data colored with labels

            ax[1].scatter(data['X'],
            data['Y'] , c = cluster_labels);
#             ax[1].scatter(centroids[:,0],centroids[:,1],
#             marker = '*' , c= 'r',s =250);
            ax[1].set_xlabel('Eruption time in mins')
            ax[1].set_ylabel('Waiting time to next eruption')
            ax[1].set_title('Visualization of clustered data', y=1.02)

            plt.suptitle(f' Silhouette analysis using num_clusters = {n_clusters}',fontsize=16,fontweight = 'semibold')
            plt.tight_layout()
            plt.savefig(f'plots/Silhouette_analysis_{n_clusters}.jpg')
  
    return stored_clusters, silh_avg_n_clusters, ax

#### Visualizing Silhouettes

In [ ]:
stored_2, avg_2, ax_2 = find_opt_clusters_4(data, new_emb, 2, 7)

In [ ]:
##acroding to the silhouette analysis, num of clusters = 2 or 3, no clear clusters
# cluster_data, cluster_id = clustering_question(data, 'emb', 4)

'''
It seems like clustering is not a good analysis technique for (this) paper dataset(s).
Next step is to experiment with BERT-based topic modeling.
'''

#only select thw desired columns to save
data = data[['title', 'abstract', 'doi', 'year', 'issue',
       'citation_info', 'authors', 'author affiliation', 'pdf_link',
       'issue_section', 'topics', 'cited times', 'keywords', 'reference', 'cluster_2', 'cluster_3',
       'cluster_4', 'cluster_5', 'cluster_6']]

data.to_excel('data with cluster.xlsx')

# Visualization

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
ax.set_xlabel('Principle Component 1', fontsize=18)
ax.set_ylabel('Principle Component 2', fontsize=18)
# ax.title.set_text('2 Component PCA for 7 Clusters', fontsize=20)
scatter = ax.scatter(data['X'], data['Y'], c=data['cluster_4'], cmap='rainbow')  #s = data['Frequency1']
plt.xticks(rotation = 0, fontsize=18)
plt.yticks(fontsize=18)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc = "upper right", fontsize=18) #, title="Clusters"
ax.add_artist(legend1)
# plt.savefig('plots/topic_cluster {}.png'.format(n), dpi = 800, bbox_inches='tight')
# plt.savefig('plots/author_cluster.png'.format(n), dpi = 800, bbox_inches='tight')
plt.savefig('plots/paper_cluster 4.png'.format(n), dpi = 800, bbox_inches='tight')
plt.show()

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [ ]:
##interactive 3d visualization
import plotly.express as px

pca = PCA(n_components=3)
pca.fit(new_emb)
x = pca.fit_transform(new_emb)

data['X1'] = x[:,0]
data['Y1'] = x[:,1]
data['Z1'] = x[:,2]

fig = px.scatter_3d(data, x='X1', y='Y1',z='Z1',
                    color="cluster_4")

fig.show()